In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

def calculate_null_zero_stats(df):
    """
    Calculate null and zero counts for each column in a DataFrame
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame to analyze
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with 'Column_Name', 'Null_Count', and 'Zero_Count' columns
    """
    # Initialize empty lists for results
    columns = []
    null_counts = []
    zero_counts = []
    
    # Iterate through all columns
    for col in df.columns:
        columns.append(col)
        null_counts.append(df[col].isna().sum())
        
        # For numeric columns, count zeros
        if pd.api.types.is_numeric_dtype(df[col]):
            zero_counts.append((df[col] == 0).sum())
        else:
            zero_counts.append(np.nan)  # Not applicable for non-numeric columns
    
    # Create results DataFrame
    results = pd.DataFrame({
        'Column_Name': columns,
        'Null_Count': null_counts,
        'Zero_Count': zero_counts
    })
    
    return results

def clean_infinite_values(df):
    """Replace infinite values with NaN, which can be handled by imputers"""
    # Replace inf and -inf with NaN
    return df.replace([np.inf, -np.inf], np.nan)

def load_and_preprocess():
    """Load and preprocess the Volve production data"""
    df = pd.read_excel("Volve data.xlsx", parse_dates=["DATEPRD"])
    
    # Analyze null and zero values
    stats_df = calculate_null_zero_stats(df)
    
    # Sort by date and reset index
    df.sort_values("DATEPRD", inplace=True)
    df.reset_index(drop=True, inplace=True)
    cols_to_keep = [
        "DATEPRD", "NPD_WELL_BORE_NAME", "WELL_BORE_CODE", "ON_STREAM_HRS",
        "Avg Downhole Press", "Avg Downhole Temp", "Avg differential pressure in tubing",
        "Avg Wellhead press", "Avg Wellhead temp", "Avg DP Choke size",
        "Bore Oil Vol", "Bore Water vol"
    ]

    df = df[cols_to_keep].copy()

    # Filter for active production periods
    df = df[df["ON_STREAM_HRS"] > 0]

    # Calculate rates
    df["oil_rate"] = df["Bore Oil Vol"] / df["ON_STREAM_HRS"]
    df["water_rate"] = df["Bore Water vol"] / df["ON_STREAM_HRS"]

    # Drop original volume columns
    df.drop(columns=["Bore Oil Vol", "Bore Water vol", "ON_STREAM_HRS"], inplace=True)

    # Filter outliers (optional: can tune 300 limit if needed)
    df = df[df["water_rate"] < 300]

    # MODIFICATION: Don't filter out rows with zero or negative pressure
    # Instead, handle them in the feature engineering and model training steps
    
    return df, stats_df

def engineer_features(well_df):
    """Create time-series features for the well data"""
    # Set date as index for time-series operations
    well_df = well_df.set_index("DATEPRD").asfreq("D")
    
    # MODIFICATION: Handle NaN values before calculating rolling means
    well_df = well_df.fillna(method='ffill').fillna(method='bfill')
    
    # Calculate rolling means
    well_df["oil_rate_ma7"] = well_df["oil_rate"].rolling(7, min_periods=1).mean()
    well_df["water_rate_ma7"] = well_df["water_rate"].rolling(7, min_periods=1).mean()
    
    # +++ ADD THIS LINE +++
    well_df = clean_infinite_values(well_df)
    
    # Create lag features
    for lag in [1, 2, 7]:
        well_df[f"oil_lag{lag}"] = well_df["oil_rate"].shift(lag)
        well_df[f"water_lag{lag}"] = well_df["water_rate"].shift(lag)
    
    # MODIFICATION: Fill NaN values in lag features using forward fill then backward fill
    well_df = well_df.fillna(method='ffill').fillna(method='bfill')
    
    # +++ ADD THIS LINE +++
    well_df = clean_infinite_values(well_df)
    
    # Reset index to return DATEPRD as a column
    well_df.reset_index(inplace=True)
    
    return well_df

def train_models(X_train, y_train):
    """Train oil and water production models"""
    # MODIFICATION: Create imputers for handling NaN values in model input
    imputer = SimpleImputer(strategy='mean')
    
    # Oil models
    xgb_oil = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('model', XGBRegressor(max_depth=10, n_estimators=500, objective="reg:squarederror", gamma=0.3, missing=np.nan,
            max_delta_step=1,
            scale_pos_weight=1))
    ])
    
    dt_oil = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('model', DecisionTreeRegressor(max_depth=8, min_samples_split=10))
    ])
    
    rf_oil = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('model', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
    ])
    
    # Water models
    lr_water = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('model', LinearRegression())
    ])
    
    pipeline_water = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', LinearRegression())
    ])
    
    # Fit models
    xgb_oil.fit(X_train, y_train["oil_rate"])
    dt_oil.fit(X_train, y_train["oil_rate"])
    rf_oil.fit(X_train, y_train["oil_rate"])
    lr_water.fit(X_train, y_train["water_rate"])
    pipeline_water.fit(X_train, y_train["water_rate"])
    
    # Return models in a dictionary
    return {
        'oil': {
            'XGBoost': xgb_oil,
            'Decision Tree': dt_oil,
            'Random Forest': rf_oil
        },
        'water': {
            'LinearRegression': lr_water,
            'Scaled → Linear': pipeline_water
        }
    }

def evaluate_models(models, X_test, y_test, well_name):
    """Evaluate model performance and generate comparison plots"""
    # Oil Model Performance Comparison
    oil_mae = []
    oil_r2 = []
    
    for name, model in models['oil'].items():
        y_pred = model.predict(X_test)
        oil_mae.append(mean_absolute_error(y_test['oil_rate'], y_pred))
        oil_r2.append(r2_score(y_test['oil_rate'], y_pred))
    
    # MAE Bar Chart
    plt.figure(figsize=(10, 6))
    plt.bar(models['oil'].keys(), oil_mae)
    plt.ylabel('MAE')
    plt.title(f'Oil Model MAE Comparison - {well_name}')
    plt.tight_layout()
    plt.savefig(f"oil_mae_comparison_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # R² Bar Chart
    plt.figure(figsize=(10, 6))
    plt.bar(models['oil'].keys(), oil_r2)
    plt.ylabel('R² Score')
    plt.title(f'Oil Model R² Comparison - {well_name}')
    plt.tight_layout()
    plt.savefig(f"oil_r2_comparison_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # Water Model Performance Comparison
    water_mae = []
    water_r2 = []
    
    for name, model in models['water'].items():
        y_pred = model.predict(X_test)
        water_mae.append(mean_absolute_error(y_test['water_rate'], y_pred))
        water_r2.append(r2_score(y_test['water_rate'], y_pred))
    
    # MAE Bar Chart
    plt.figure(figsize=(10, 6))
    plt.bar(models['water'].keys(), water_mae)
    plt.ylabel('MAE')
    plt.title(f'Water Model MAE Comparison - {well_name}')
    plt.tight_layout()
    plt.savefig(f"water_mae_comparison_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # R² Bar Chart
    plt.figure(figsize=(10, 6))
    plt.bar(models['water'].keys(), water_r2)
    plt.ylabel('R² Score')
    plt.title(f'Water Model R² Comparison - {well_name}')
    plt.tight_layout()
    plt.savefig(f"water_r2_comparison_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # Return the best models based on R2 score
    best_oil_model = list(models['oil'].keys())[np.argmax(oil_r2)]
    best_water_model = list(models['water'].keys())[np.argmax(water_r2)]
    
    return {
        'oil': models['oil'][best_oil_model],
        'water': models['water'][best_water_model],
        'metrics': {
            'oil': {'model': best_oil_model, 'r2': max(oil_r2), 'mae': oil_mae[np.argmax(oil_r2)]},
            'water': {'model': best_water_model, 'r2': max(water_r2), 'mae': water_mae[np.argmax(water_r2)]}
        }
    }

def recursive_forecast(model_oil, model_water, hist_df, days, feature_cols, static_cols):
    """Generate recursive forecasts for oil and water production"""
    # Make a copy and ensure it's indexed by DATEPRD
    hist = hist_df.set_index("DATEPRD").sort_index().copy()
    
    # MODIFICATION: Fill any NaN values in the historical data
    hist = hist.fillna(method='ffill').fillna(method='bfill')
    hist = clean_infinite_values(hist)

    # Prepare future date index
    future_dates = pd.date_range(
        start=hist.index[-1] + pd.Timedelta(days=1),
        periods=days,
        freq="D"
    )

    preds = []
    for day in future_dates:
        # Build a single-row dict of features
        X_day = {}

        # Static features = carry last known
        for col in static_cols:
            X_day[col] = hist[col].iloc[-1]

        # Rolling-means
        X_day["oil_rate_ma7"] = hist["oil_rate"].rolling(7, min_periods=1).mean().iloc[-1]
        X_day["water_rate_ma7"] = hist["water_rate"].rolling(7, min_periods=1).mean().iloc[-1]

        # Lag features
        for lag in [1, 2, 7]:
            X_day[f"oil_lag{lag}"] = hist["oil_rate"].shift(lag).fillna(method='ffill').iloc[-1]
            X_day[f"water_lag{lag}"] = hist["water_rate"].shift(lag).fillna(method='ffill').iloc[-1]

        # Turn into a DataFrame & keep only the columns your models expect
        X_day_df = pd.DataFrame([X_day], index=[day])[feature_cols]

        # Predict
        oil_p = model_oil.predict(X_day_df)[0]
        water_p = model_water.predict(X_day_df)[0]

        # Store predictions
        preds.append((day, oil_p, water_p))

        # Append forecasts back to history for the next iteration
        hist.loc[day, "oil_rate"] = oil_p
        hist.loc[day, "water_rate"] = water_p
        for col in static_cols:
            hist.loc[day, col] = X_day_df[col].iloc[0]

    # Build result DataFrame
    future = pd.DataFrame(preds, columns=["DATEPRD", "oil_pred", "water_pred"])
    future.set_index("DATEPRD", inplace=True)
    
    # Calculate volumes and cumulative values
    future["oil_volume"] = future["oil_pred"] * 24
    future["water_volume"] = future["water_pred"] * 24
    future["cum_oil"] = future["oil_volume"].cumsum()
    future["cum_water"] = future["water_volume"].cumsum()
    
    return future

def forecast_with_uncertainty(well_name, future, residuals_oil, residuals_water, n_samples=100):
    """Generate forecast with bootstrapped confidence intervals"""
    oil_pred = future["oil_pred"].values
    water_pred = future["water_pred"].values
    
    boot_oil = []
    boot_water = []
    for _ in range(n_samples):
        boot_oil.append(oil_pred + np.random.choice(residuals_oil, size=len(oil_pred), replace=True))
        boot_water.append(water_pred + np.random.choice(residuals_water, size=len(water_pred), replace=True))

    boot_oil = np.array(boot_oil)
    boot_water = np.array(boot_water)

    # 90% Confidence Intervals
    lower_oil = np.percentile(boot_oil, 5, axis=0)
    upper_oil = np.percentile(boot_oil, 95, axis=0)

    lower_water = np.percentile(boot_water, 5, axis=0)
    upper_water = np.percentile(boot_water, 95, axis=0)

    # Plot forecast with CI bands
    plt.figure(figsize=(12, 6))
    plt.plot(future.index, oil_pred, label="Oil Forecast")
    plt.fill_between(future.index, lower_oil, upper_oil, alpha=0.2, label="Oil 90% CI")
    plt.plot(future.index, water_pred, label="Water Forecast")
    plt.fill_between(future.index, lower_water, upper_water, alpha=0.2, label="Water 90% CI")
    plt.title(f"Forecast with Bootstrapped 90% Confidence Intervals - {well_name}")
    plt.xlabel("Date")
    plt.ylabel("Production Rate")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"forecast_ci_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # Add CI bounds to the future dataframe
    future["oil_lower"] = lower_oil
    future["oil_upper"] = upper_oil
    future["water_lower"] = lower_water
    future["water_upper"] = upper_water
    
    return future

def visualize_forecasts(well_name, well_df, future):
    """Create visualization for the forecasts"""
    # Production rates forecast
    plt.figure(figsize=(12, 6))
    plt.plot(well_df["DATEPRD"], well_df["oil_rate"], label="Historical Oil Rate")
    plt.plot(future.index, future["oil_pred"], "--", label="Forecasted Oil Rate")
    plt.plot(well_df["DATEPRD"], well_df["water_rate"], label="Historical Water Rate")
    plt.plot(future.index, future["water_pred"], "--", label="Forecasted Water Rate")
    plt.xlabel("Date")
    plt.ylabel("Production Rate")
    plt.title(f"180-Day Oil & Water Production Forecast - {well_name}")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(f"forecast_{well_name.replace('/', '_')}.png")
    plt.close()
    
    # Cumulative forecast
    plt.figure(figsize=(10, 5))
    plt.plot(future.index, future["cum_oil"], label="Cumulative Oil Forecast")
    plt.plot(future.index, future["cum_water"], label="Cumulative Water Forecast")
    plt.title(f"180-Day Cumulative Oil & Water Volume Forecast - {well_name}")
    plt.xlabel("Date")
    plt.ylabel("Volume")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"cumulative_forecast_{well_name.replace('/', '_')}.png")
    plt.close()

def simulate_choke_scenarios(well_name, model_oil, base_df, choke_values, feature_cols, static_cols, days=180):
    """Simulate different choke size scenarios"""
    scenario_results = {}
    
    for choke in choke_values:
        # Create future DataFrame for each choke scenario
        future_scenario = pd.DataFrame(
            index=pd.date_range(base_df["DATEPRD"].max() + pd.Timedelta(days=1), periods=days, freq="D")
        )
        
        # Static features: either override choke or copy last known value
        for col in static_cols:
            if col == "Avg DP Choke size":  # Fixed column name to match actual data
                future_scenario[col] = choke
            else:
                future_scenario[col] = base_df[col].iloc[-1]
        
        # Rolling means
        future_scenario["oil_rate_ma7"] = base_df["oil_rate"].rolling(7, min_periods=1).mean().iloc[-1]
        future_scenario["water_rate_ma7"] = base_df["water_rate"].rolling(7, min_periods=1).mean().iloc[-1]
        
        # Lag features
        for lag in [1, 2, 7]:
            future_scenario[f"oil_lag{lag}"] = base_df["oil_rate"].shift(lag).fillna(method='ffill').iloc[-1]
            future_scenario[f"water_lag{lag}"] = base_df["water_rate"].shift(lag).fillna(method='ffill').iloc[-1]
        
        # Select feature columns
        X_scenario = future_scenario[feature_cols]
        
        # Predict for the scenario
        scenario_results[choke] = model_oil.predict(X_scenario)

    # Plot all scenarios together
    plt.figure(figsize=(10, 5))
    for choke, preds in scenario_results.items():
        plt.plot(future_scenario.index, preds, label=f"Choke = {choke} mm")
    plt.title(f"Oil Rate Forecast Under Varying Choke Sizes - {well_name}")
    plt.xlabel("Date")
    plt.ylabel("Oil Rate")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"choke_scenarios_{well_name.replace('/', '_')}.png")
    plt.close()
    
    return scenario_results

def export_forecast(well_name, future):
    """Export the forecast to Excel for a single well"""
    # Reset the index, which brings 'DATEPRD' back as a column
    export_df = future.reset_index()
    
    # Rename the DATEPRD column to Date
    export_df = export_df.rename(columns={"DATEPRD": "Date"})
    
    # Select and rename the forecast columns
    export_df = export_df[["Date", "oil_pred", "water_pred", "cum_oil", "cum_water", 
                          "oil_lower", "oil_upper", "water_lower", "water_upper"]]
    export_df.columns = ["Date", "Oil Forecast", "Water Forecast", "Cumulative Oil", "Cumulative Water",
                        "Oil Lower CI", "Oil Upper CI", "Water Lower CI", "Water Upper CI"]
    
    # Export to Excel
    export_df.to_excel(f"Oil_Water_180Day_Forecast_{well_name.replace('/', '_')}.xlsx", index=False)
    print(f"Forecast data for {well_name} exported to 'Oil_Water_180Day_Forecast_{well_name.replace('/', '_')}.xlsx'")
    
    return export_df

def main():
    # Load and preprocess data
    df, stats_df = load_and_preprocess()
    
    # Define feature columns
    static_cols = [
        "Avg Downhole Press",
        "Avg Downhole Temp",
        "Avg differential pressure in tubing",
        "Avg Wellhead press",
        "Avg Wellhead temp",
        "Avg DP Choke size",
    ]

    feature_cols = static_cols + [
        "oil_rate_ma7", "water_rate_ma7",
        "oil_lag1", "oil_lag2", "oil_lag7",
        "water_lag1", "water_lag2", "water_lag7",
    ]
    
    # Get unique wells
    wells = df["NPD_WELL_BORE_NAME"].unique()
    well_forecasts = {}
    well_metrics = {}
    
    # Create a list to collect data quality stats for each well
    well_data_quality = []
    
    # Loop through each well
    for well_name in wells:
        try:
            print(f"\nProcessing well: {well_name}")
            
            # Filter data for the current well
            well_df = df[df["NPD_WELL_BORE_NAME"] == well_name].copy()
            
            # Calculate null and zero stats for this well
            well_stats = calculate_null_zero_stats(well_df)
            well_stats['well_name'] = well_name
            well_data_quality.append(well_stats)
            
            # Skip wells with insufficient data
            if len(well_df) < 30:
                print(f"Skipping {well_name} due to insufficient data points (only {len(well_df)} rows)")
                continue
            
            # Engineer features
            well_df = engineer_features(well_df)
            
            # Skip wells with insufficient data after feature engineering
            if len(well_df) < 20:
                print(f"Skipping {well_name} due to insufficient data after feature engineering (only {len(well_df)} rows)")
                continue
            
            # Prepare data for modeling
            X = well_df[feature_cols]
            y = well_df[["oil_rate", "water_rate"]]
            X = clean_infinite_values(X)
            y = clean_infinite_values(y)
            
            # Train/Test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
            
            # Train models
            models = train_models(X_train, y_train)
            
            # Evaluate models
            best_models = evaluate_models(models, X_test, y_test, well_name)
            well_metrics[well_name] = best_models['metrics']
            
            # Calculate residuals for uncertainty quantification
            oil_residuals = y_test["oil_rate"] - best_models['oil'].predict(X_test)
            water_residuals = y_test["water_rate"] - best_models['water'].predict(X_test)
            
            # Generate forecasts
            future = recursive_forecast(best_models['oil'], best_models['water'], 
                                       well_df, days=180, feature_cols=feature_cols, 
                                       static_cols=static_cols)
            
            # Add uncertainty bounds
            future = forecast_with_uncertainty(well_name, future, oil_residuals, water_residuals)
            
            # Visualize forecasts
            visualize_forecasts(well_name, well_df, future)
            
            # Simulate choke scenarios
            choke_scenarios = simulate_choke_scenarios(well_name, best_models['oil'], well_df, 
                                                      choke_values=[10, 20, 30], 
                                                      feature_cols=feature_cols,
                                                      static_cols=static_cols)
            
            # Export single well forecast
            export_forecast(well_name, future)
            
            # Store forecast for consolidation
            future['well_name'] = well_name
            well_forecasts[well_name] = future
            
            print(f"Completed processing for well: {well_name}")
            
        except Exception as e:
            print(f"Error processing well {well_name}: {str(e)}")
    
    # Consolidated forecast visualization
    plt.figure(figsize=(15, 8))
    for well_name, future_data in well_forecasts.items():
        plt.plot(future_data.index, future_data["oil_pred"], label=f"{well_name} - Oil")
    
    plt.xlabel("Date")
    plt.ylabel("Oil Production Rate")
    plt.title("180-Day Oil Production Forecast - All Wells")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig("all_wells_oil_forecast.png")
    plt.close()
    
    # Water Production Forecast - All Wells
    plt.figure(figsize=(15, 8))
    for well_name, future_data in well_forecasts.items():
        plt.plot(future_data.index, future_data["water_pred"], label=f"{well_name} - Water")
    
    plt.xlabel("Date")
    plt.ylabel("Water Production Rate")
    plt.title("180-Day Water Production Forecast - All Wells")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig("all_wells_water_forecast.png")
    plt.close()
    
    # Export consolidated forecasts for all wells
    if well_forecasts:  # Check if any forecasts were generated
        consolidated_forecasts = pd.DataFrame()
        
        for well_name, future_data in well_forecasts.items():
            well_data = future_data.reset_index().copy()
            well_data["well_name"] = well_name
            
            # Add model performance metrics
            if well_name in well_metrics:
                well_data["oil_model"] = well_metrics[well_name]['oil']['model']
                well_data["oil_r2"] = well_metrics[well_name]['oil']['r2']
                well_data["oil_mae"] = well_metrics[well_name]['oil']['mae']
                well_data["water_model"] = well_metrics[well_name]['water']['model']
                well_data["water_r2"] = well_metrics[well_name]['water']['r2']
                well_data["water_mae"] = well_metrics[well_name]['water']['mae']
            
            consolidated_forecasts = pd.concat([consolidated_forecasts, well_data], ignore_index=True)
        
        # Combine well data quality statistics
        combined_well_stats = pd.concat(well_data_quality, ignore_index=True) if well_data_quality else pd.DataFrame()
        
        # Create a writer object to save multiple sheets
        with pd.ExcelWriter("All_Wells_180Day_Forecast.xlsx") as writer:
            consolidated_forecasts.to_excel(writer, sheet_name="Forecasts", index=False)
            
            # Add the null/zero stats to the Excel file
            stats_df.to_excel(writer, sheet_name="Dataset_Stats", index=False)
            
            if not combined_well_stats.empty:
                combined_well_stats.to_excel(writer, sheet_name="Well_Data_Quality", index=False)
        
        print("\nConsolidated forecast data exported to 'All_Wells_180Day_Forecast.xlsx'")
        
        # Additionally, create a summary metrics file
        if well_metrics:
            metrics_df = pd.DataFrame()
            for well_name, metrics in well_metrics.items():
                well_row = {
                    'well_name': well_name,
                    'oil_model': metrics['oil']['model'],
                    'oil_r2': metrics['oil']['r2'],
                    'oil_mae': metrics['oil']['mae'],
                    'water_model': metrics['water']['model'],
                    'water_r2': metrics['water']['r2'],
                    'water_mae': metrics['water']['mae']
                }
                metrics_df = pd.concat([metrics_df, pd.DataFrame([well_row])], ignore_index=True)
            
            metrics_df.to_excel("Well_Model_Performance_Summary.xlsx", index=False)
            print("Model performance summary exported to 'Well_Model_Performance_Summary.xlsx'")
    else:
        print("No well forecasts were generated. Check your data or model parameters.")

if __name__ == "__main__":
    main()


Processing well: 15/9-F-12
Forecast data for 15/9-F-12 exported to 'Oil_Water_180Day_Forecast_15_9-F-12.xlsx'
Completed processing for well: 15/9-F-12

Processing well: 15/9-F-4
Error processing well 15/9-F-4: Input X contains infinity or a value too large for dtype('float32').

Processing well: 15/9-F-14
Forecast data for 15/9-F-14 exported to 'Oil_Water_180Day_Forecast_15_9-F-14.xlsx'
Completed processing for well: 15/9-F-14

Processing well: 15/9-F-5
Error processing well 15/9-F-5: Input X contains infinity or a value too large for dtype('float32').

Processing well: 15/9-F-11
Forecast data for 15/9-F-11 exported to 'Oil_Water_180Day_Forecast_15_9-F-11.xlsx'
Completed processing for well: 15/9-F-11

Processing well: 15/9-F-15 D
Forecast data for 15/9-F-15 D exported to 'Oil_Water_180Day_Forecast_15_9-F-15 D.xlsx'
Completed processing for well: 15/9-F-15 D

Processing well: 15/9-F-1 C
Forecast data for 15/9-F-1 C exported to 'Oil_Water_180Day_Forecast_15_9-F-1 C.xlsx'
Completed proc